In [1]:
# Зависимости
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import os

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

from sklearn.neural_network import MLPRegressor, MLPClassifier

from sklearn.metrics import mean_squared_error, f1_score

import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout

2022-03-09 04:35:58.877629: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-03-09 04:35:58.877684: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
# Инициализируем все известные генераторы случайных чисел / Setting all known random seeds
my_code = "Магдиев"
seed_limit = 2 ** 32
my_seed = int.from_bytes(my_code.encode(), "little") % seed_limit

os.environ['PYTHONHASHSEED']=str(my_seed)

random.seed(my_seed)

np.random.seed(my_seed)

tf.compat.v1.set_random_seed(my_seed)

session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
tf.compat.v1.keras.backend.set_session(sess)

2022-03-09 04:36:00.966000: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-03-09 04:36:00.966684: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-03-09 04:36:00.966729: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-03-09 04:36:00.966757: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (jupyter-mar1eena-2dnotebook-5ffor-5fai-5fmain-2dahj34494): /proc/driver/nvidia/version does not exist


In [3]:
# Читаем данные из файла
data = pd.read_csv("datasets/Fish.csv")

In [4]:
data

,Species,Weight,Length1,Length2,Length3,Height,Width
0,Bream,242.0,23.2,25.4,30.0,11.5200,4.0200
1,Bream,290.0,24.0,26.3,31.2,12.4800,4.3056
2,Bream,340.0,23.9,26.5,31.1,12.3778,4.6961
3,Bream,363.0,26.3,29.0,33.5,12.7300,4.4555
4,Bream,430.0,26.5,29.0,34.0,12.4440,5.1340
...,...,...,...,...,...,...,...
154,Smelt,12.2,11.5,12.2,13.4,2.0904,1.3936
155,Smelt,13.4,11.7,12.4,13.5,2.4300,1.2690
156,Smelt,12.2,12.1,13.0,13.8,2.2770,1.2558
157,Smelt,19.7,13.2,14.3,15.2,2.8728,2.0672


In [5]:
# Определим размер валидационной и тестовой выборок
val_test_size = round(0.2*len(data))
print(val_test_size)

32


In [6]:
# Создадим обучающую, валидационную и тестовую выборки
random_state = my_seed
train_val, test = train_test_split(data, test_size=val_test_size, random_state=random_state)
train, val = train_test_split(train_val, test_size=val_test_size, random_state=random_state)
print(len(train), len(val), len(test))

95 32 32


In [7]:
# Значения в числовых столбцах преобразуем к отрезку [0,1].
# Для настройки скалировщика используем только обучающую выборку.
num_columns = ['Weight', 'Length1', 'Length2', 'Length3', 'Height', 'Width']
ord_columns = ['Species']

ct = ColumnTransformer(transformers=[
    ('numerical', MinMaxScaler(), num_columns), 
    ('ordinal', OneHotEncoder(), ord_columns)])

ct.fit(train)

ColumnTransformer(transformers=[('numerical', MinMaxScaler(),
                                 ['Weight', 'Length1', 'Length2', 'Length3',
                                  'Height', 'Width']),
                                ('ordinal', OneHotEncoder(), ['Species'])])

In [8]:
# Преобразуем значения, тип данных приводим к DataFrame
sc_train = pd.DataFrame(ct.transform(train))
sc_test = pd.DataFrame(ct.transform(test))
sc_val = pd.DataFrame(ct.transform(val))

In [9]:
# Устанавливаем названия столбцов
column_names = num_columns + list(range(7))
sc_train.columns = column_names
sc_test.columns = column_names
sc_val.columns = column_names

In [10]:
sc_train

,Weight,Length1,Length2,Length3,Height,Width,0,1,2,3,4,5,6
0,0.116129,0.331959,0.325581,0.346014,0.311006,0.402909,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,0.467742,0.501031,0.515504,0.581522,0.849262,0.705571,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.109032,0.298969,0.302326,0.333333,0.336998,0.392930,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0.294194,0.670103,0.660853,0.664855,0.322232,0.461618,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,0.193548,0.400000,0.393411,0.385870,0.339946,0.505723,0.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
90,0.174194,0.342268,0.350775,0.371377,0.372462,0.451187,0.0,0.0,0.0,0.0,0.0,0.0,1.0
91,0.251613,0.453608,0.451550,0.474638,0.450217,0.607155,0.0,0.0,0.0,0.0,1.0,0.0,0.0
92,0.438710,0.501031,0.515504,0.576087,0.797523,0.716481,1.0,0.0,0.0,0.0,0.0,0.0,0.0
93,0.529032,0.610309,0.612403,0.610507,0.545987,0.786874,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [11]:
# В качестве входных параметров используем все числовые параметры
# в качестве выходного - единственный категориальный параметр.
x_labels = num_columns[:]
y_labels = column_names[6:]
print(x_labels)
print(y_labels)

['Weight', 'Length1', 'Length2', 'Length3', 'Height', 'Width']
[0, 1, 2, 3, 4, 5, 6]


In [12]:
# Отберем необходимые параметры
x_train = sc_train[x_labels]
x_test = sc_test[x_labels]
x_val = sc_val[x_labels]

y_train = sc_train[y_labels]
y_test = sc_test[y_labels]
y_val = sc_val[y_labels]


In [13]:
y_train

,0,1,2,3,4,5,6
0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...
90,0.0,0.0,0.0,0.0,0.0,0.0,1.0
91,0.0,0.0,0.0,0.0,1.0,0.0,0.0
92,1.0,0.0,0.0,0.0,0.0,0.0,0.0
93,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [14]:
# Задание №1 - решение задачи классификации.
# В качестве входных параметров используем все числовые параметры,
# в качестве выходного - единственный категориальный параметр.

In [15]:
# Создайте нейронную сеть для решения задачи классификации двумя способами: с помощью sklearn и keras.
# Сравните их эффективность.
# Для keras используйте loss и metrics = 'categorical_crossentropy'.

In [16]:
# Создадим нейроннную сеть для решения задачи классификации на базе библиотеки sklearn
# Выбираем количество итераций 10000 для оптимизации
reg = MLPClassifier(alpha=0.0, batch_size=16, epsilon=1e-07, max_iter=10000)
reg.get_params()

{'activation': 'relu',
 'alpha': 0.0,
 'batch_size': 16,
 'beta_1': 0.9,
 'beta_2': 0.999,
 'early_stopping': False,
 'epsilon': 1e-07,
 'hidden_layer_sizes': (100,),
 'learning_rate': 'constant',
 'learning_rate_init': 0.001,
 'max_fun': 15000,
 'max_iter': 10000,
 'momentum': 0.9,
 'n_iter_no_change': 10,
 'nesterovs_momentum': True,
 'power_t': 0.5,
 'random_state': None,
 'shuffle': True,
 'solver': 'adam',
 'tol': 0.0001,
 'validation_fraction': 0.1,
 'verbose': False,
 'warm_start': False}

In [17]:
# Обучим нейронную сеть
reg.fit(x_train, y_train)

MLPClassifier(alpha=0.0, batch_size=16, epsilon=1e-07, max_iter=10000)

In [18]:
pred_val1 = reg.predict(x_val)


In [19]:
# Проверим работу обученной нейронной сети на валидационной выборке

mse1 = mean_squared_error(y_val, pred_val1)
print(mse1)

0.026785714285714284


In [20]:
# Создадим нейронную сеть на базе библиотеки keras
model = Sequential()

model.add(Dense(7, input_dim=6, activation='sigmoid', use_bias=False))

model.add(Dense(7,input_dim=6, activation='sigmoid', use_bias=False))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['categorical_crossentropy'])

model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 7)                 42        
                                                                 
 dense_1 (Dense)             (None, 7)                 49        
                                                                 
Total params: 91
Trainable params: 91
Non-trainable params: 0
_________________________________________________________________


In [21]:
# Обучим нейронную сеть
history1 = model.fit(x_train, y_train
                    , validation_data = (x_val, y_val)
                    , epochs=30, batch_size=16, verbose=10)

Epoch 1/30
Epoch 2/30
Epoch 3/30
Epoch 4/30
Epoch 5/30
Epoch 6/30
Epoch 7/30
Epoch 8/30
Epoch 9/30
Epoch 10/30
Epoch 11/30
Epoch 12/30
Epoch 13/30
Epoch 14/30
Epoch 15/30
Epoch 16/30
Epoch 17/30
Epoch 18/30
Epoch 19/30
Epoch 20/30
Epoch 21/30
Epoch 22/30
Epoch 23/30
Epoch 24/30
Epoch 25/30
Epoch 26/30
Epoch 27/30
Epoch 28/30
Epoch 29/30
Epoch 30/30


In [22]:
pred_val2 = model.predict(x_val)
mse2 = mean_squared_error(y_val, pred_val2)
print(mse2)

0.24409581822826973


In [23]:
# Задание №2 - использование dropout-слоя.

In [24]:
# Создайте нейронную сеть для решения задачи классификации с помощью keras. Используйте dropout-слой.
# Сравните эффективность нейронных сетей с dropout-слоем и без него.
# Попробуйте найти такие параметры dropout-слоя, чтобы сеть с ним работала лучше, чем без него.

In [25]:
drop = Sequential()

drop.add(Dense(100, input_dim=6, activation='relu', use_bias=False))

drop.add(Dropout(rate=0.5))

drop.add(Dense(7, activation='linear', use_bias=False))

drop.compile(loss='mean_squared_error', optimizer='adam', metrics=['mean_squared_error'])

drop.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 100)               600       
                                                                 
 dropout (Dropout)           (None, 100)               0         
                                                                 
 dense_3 (Dense)             (None, 7)                 700       
                                                                 
Total params: 1,300
Trainable params: 1,300
Non-trainable params: 0
_________________________________________________________________


In [26]:
history2 = drop.fit(x_train, y_train, validation_data = (x_val, y_val), 
                    epochs=50, batch_size=16, verbose=5)

Epoch 1/50
Epoch 2/50
Epoch 3/50
Epoch 4/50
Epoch 5/50
Epoch 6/50
Epoch 7/50
Epoch 8/50
Epoch 9/50
Epoch 10/50
Epoch 11/50
Epoch 12/50
Epoch 13/50
Epoch 14/50
Epoch 15/50
Epoch 16/50
Epoch 17/50
Epoch 18/50
Epoch 19/50
Epoch 20/50
Epoch 21/50
Epoch 22/50
Epoch 23/50
Epoch 24/50
Epoch 25/50
Epoch 26/50
Epoch 27/50
Epoch 28/50
Epoch 29/50
Epoch 30/50
Epoch 31/50
Epoch 32/50
Epoch 33/50
Epoch 34/50
Epoch 35/50
Epoch 36/50
Epoch 37/50
Epoch 38/50
Epoch 39/50
Epoch 40/50
Epoch 41/50
Epoch 42/50
Epoch 43/50
Epoch 44/50
Epoch 45/50
Epoch 46/50
Epoch 47/50
Epoch 48/50
Epoch 49/50
Epoch 50/50


In [27]:
pred_val3 = drop.predict(x_val)
mse3 = mean_squared_error(y_val, pred_val3)
print(mse3)

0.08383904030574461


In [28]:
print(mse1)
print(mse2)
print(mse3)

0.026785714285714284
0.24409581822826973
0.08383904030574461
